In [1]:
import os
import requests
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from dotenv import load_dotenv

load_dotenv()

# トークンを取得する関数
def get_token():
    auth_url = os.getenv('AICORE_AUTH_URL')
    client_id = os.getenv('AICORE_CLIENT_ID')
    client_secret = os.getenv('AICORE_CLIENT_SECRET')

    token_url = f"{auth_url}/oauth/token"
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    response = requests.post(token_url, data=data)
    response.raise_for_status()

    token = response.json().get('access_token')
    if not token:
        raise ValueError("トークンが取得できませんでした。")

    return token

# トークンを取得
token = get_token()
print(f'Token: {token}')

# プロキシクライアントの取得
proxy_client = get_proxy_client('gen-ai-hub', token=token)

# デプロイメント一覧を取得
deployments_list = proxy_client.deployments
print("model_name, deployment_id, config_name")
for deployment in deployments_list:
    print(f"{deployment.model_name}, {deployment.deployment_id}, {deployment.config_name}")

Token: eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vZXUxLmF1dGhlbnRpY2F0aW9uLmV1MTAuaGFuYS5vbmRlbWFuZC5jb20vdG9rZW5fa2V5cyIsImtpZCI6Im15LWp3dC1rZXktMSIsInR5cCI6IkpXVCIsImppZCI6ICJJeGMzajhvM0ZGdTR2QlNsMEwrcktnYTBNdE02NXMvRW4yUmRQV2pFdEZnPSJ9.eyJqdGkiOiIzYTk4ZmI1NGI4Y2Y0ZDk0ODFiZTM5MGI2ZjRjNTE3ZCIsImV4dF9hdHRyIjp7ImVuaGFuY2VyIjoiWFNVQUEiLCJzdWJhY2NvdW50aWQiOiIxMDgwNmFkZC0zZWZkLTQ5NWMtODYxYi1iNzU5ZTQ4YjhiMTYiLCJ6ZG4iOiJldTEiLCJzZXJ2aWNlaW5zdGFuY2VpZCI6IjFmZGUwNWYwLWU0YWItNGJlMC1iMWU0LWJkZjA4ZjQ5OTY3OSJ9LCJzdWIiOiJzYi0xZmRlMDVmMC1lNGFiLTRiZTAtYjFlNC1iZGYwOGY0OTk2NzkhYjQ3OTB8YWljb3JlIWI1NDAiLCJhdXRob3JpdGllcyI6WyJhaWNvcmUhYjU0MC5kb2NrZXJyZWdpc3RyeXNlY3JldC5jcmVkZW50aWFscy51cGRhdGUiLCJhaWNvcmUhYjU0MC5tbGZjb25uZWN0aW9uLmNyZWRlbnRpYWxzLmNyZWF0ZSIsImFpY29yZSFiNTQwLmRvY2tlcnJlZ2lzdHJ5c2VjcmV0LmNyZWRlbnRpYWxzLmRlbGV0ZSIsImFpY29yZSFiNTQwLnJlcG9zaXRvcmllcy5yZWFkIiwiYWljb3JlIWI1NDAuZG9ja2VycmVnaXN0cnlzZWNyZXQuY3JlZGVudGlhbHMucmVhZCIsImFpY29yZSFiNTQwLnNjZW5hcmlvcy5leGVjdXRpb25zY2hlZHVsZXMuY3JlYXRlIiwiYWlj

In [2]:
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings

# 取得したデプロイメントIDを使用
deployment_id = "d8b1e5eb7341b1f3"

# embedding_modelの設定
embedding_model = OpenAIEmbeddings(deployment_id=deployment_id, proxy_client=proxy_client)

In [3]:
import os
from hdbcli import dbapi
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import HanaDB
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()

def get_embedding(input) -> str:
    response = embedding_model.embed_query(input)
    return response

# データベースの接続と初期化
def connect_database():
    connection = dbapi.connect(
        address=os.getenv("HANA_DB_ADDRESS"),
        port=os.getenv("HANA_DB_PORT"),
        user=os.getenv("HANA_DB_USER"),
        password=os.getenv("HANA_DB_PASSWORD"),
        autocommit=True,
        sslValidateCertificate=False,
    )
    db = HanaDB(
        connection=connection,
        embedding=embedding_model,
        table_name="FILE_EMBEDDINGS",
    )
    db.delete(filter={})  # データベースの初期化
    return connection, db

# JSONファイルを処理してDBに格納
def process_json_files(json_folder, db):
    for file_name in os.listdir(json_folder):
        file_path = os.path.join(json_folder, file_name)
        loader = JSONLoader(file_path, jq_schema='.page_content', text_content=False)
        documents = loader.load()
        for doc in documents:
            embedding = get_embedding(doc.page_content)
            doc.metadata['embedding'] = embedding
        db.add_documents(documents)
        print(f"Processed and added JSON file: {file_path}")

# メインフロー
def main():
    json_folder = "src/all_JSONs"
    connection, db = connect_database()
    process_json_files(json_folder, db)
    connection.close()

# Jupyter Notebookで実行
if __name__ == "__main__":
    main()

Processed and added JSON file: src/all_JSONs/BTP 2021 2022 Sprint.json
Processed and added JSON file: src/all_JSONs/BTP All IN Program.json
Processed and added JSON file: src/all_JSONs/BTP Jam Blog Post Migration.json
Processed and added JSON file: src/all_JSONs/BTP Preferred Success L1 for BTP CSPs.json
Processed and added JSON file: src/all_JSONs/BTP, Analytics Cloud_ Service Map - Ad Hoc Advisory.json
Processed and added JSON file: src/all_JSONs/BTP, Analytics Cloud_ Service Map - Development Infrastructure & Administrative Readiness.json
Processed and added JSON file: src/all_JSONs/BTP, Analytics Cloud_ Service Map - New Feature Adoption Guidance.json
Processed and added JSON file: src/all_JSONs/BTP, Analytics Cloud_ Service Map - System Administrator & End User Training.json
Processed and added JSON file: src/all_JSONs/BTP, Analytics Cloud_ Service Maps - Success Check_ Business Goals & KPI Reporting.json
Processed and added JSON file: src/all_JSONs/BTP, Integration Suite_ Service